# This serves as a prototype for calling, cleaning, and aggregating data from Riot's API.
### My API key is private, but you can easily create a .env for yourself and store it here.

In [1]:
# Import Dependencies
from dotenv import load_dotenv
import os
import json
import pandas as pd
import requests
import urllib.parse
from pprint import pprint

# Load environment variables from the .env file
load_dotenv()

# Retrieve the Riot API key
riot_api_key = os.getenv("RIOT_API_KEY")

# Riot API base URL
base_url = "https://americas.api.riotgames.com"

In [2]:
# Grab a Summoner's puuid

def get_puuid(game_name, tag_line):
    """
    Get the PUUID (Player Unique ID) for a given summoner using their Riot ID (gameName and tagLine).
    
    Parameters:
    - game_name (str): Summoner's game name (e.g., 'SummonerOne')
    - tag_line (str): Summoner's tag line (e.g., '0xZZk')
    
    Returns:
    - str: The PUUID of the summoner, or None if the request fails.
    """

    # Endpoint to get account info by Riot ID
    endpoint = f"/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    
    # Construct the full URL
    url = base_url + endpoint
    
    # Define headers, including the API Key
    headers = {
        "X-Riot-Token": riot_api_key
    }
    
    # Make the API request
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract PUUID from the response
        return data.get("puuid", None)
    else:
        print(f"Error fetching PUUID: {response.status_code} - {response.text}")
        return None

In [3]:
# Testing my summoner name
unearthed_puuid = get_puuid('Unearthed','0xZZk')

In [4]:
# Grab Summoners last n matches

def get_match_ids(puuid, count=10):
    headers = {
        "X-Riot-Token": riot_api_key
    }
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    params = {
        "start": 0,
        "count": count
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()  # List of match IDs
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [5]:
# My last ten matches - 2 were remakes and will be removed.
match_ids = get_match_ids(unearthed_puuid,12)
match_ids

['NA1_5282347493',
 'NA1_5282327716',
 'NA1_5282314897',
 'NA1_5282303352',
 'NA1_5282291678',
 'NA1_5282291830',
 'NA1_5282279999',
 'NA1_5282259071',
 'NA1_5282256473',
 'NA1_5282245548',
 'NA1_5281660396',
 'NA1_5281653581']

In [11]:
#Get match metadata for a matchId

def get_match_metadata(match_id):
    """
    Fetches match metadata for a given match ID using the base_url.
    Includes game duration and participant stats.
    """
    url = f"{base_url}/lol/match/v5/matches/{match_id}"
    headers = {
        "X-Riot-Token": riot_api_key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"[ERROR] Match metadata fetch failed for {match_id}: {response.status_code}")
        return None

In [7]:
#Get match timeline data by match id.

def get_timeline_data(matchId):
    
    endpoint = f"/lol/match/v5/matches/{matchId}/timeline"


    headers = {
        "X-Riot-Token": riot_api_key
    }

    url = base_url + endpoint


    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()  # Timeline data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [14]:
#Send this to a JSON to communicate with ChatGPT.

with open("data/prototype-data/test_data.json", "w") as f:
        json.dump(match_json, f, indent=4)
print("Saved successfully.")

Saved successfully.


In [12]:
#Find the matches with non - remake data.
trimmed_ids = []
for gameId in match_ids:
    metaData = get_match_metadata(gameId)

    if metaData["info"]["gameDuration"] > 240:
        trimmed_ids.append(gameId)
    else:
        print(f"{gameId} associated with a game that was remade.")

NA1_5282291830 associated with a game that was remade.
NA1_5282256473 associated with a game that was remade.


In [ ]:
#Hardcoded game ids:
trimmed_ids = ['NA1_5282347493',
 'NA1_5282327716',
 'NA1_5282314897',
 'NA1_5282303352',
 'NA1_5282291678',
 'NA1_5282279999',
 'NA1_5282259071',
 'NA1_5282245548',
 'NA1_5281660396',
 'NA1_5281653581']

In [16]:
#Get total data for each game.
metaData_JSONS = []
gameData_JSONS = []

for little_id in trimmed_ids:
    metaData_JSONS.append(get_match_metadata(little_id))
    gameData_JSONS.append(get_timeline_data(little_id))



## Above is considered what will be riot_api.py. We will get the player id, match id, then meta and game data. Below is going to be etl.py where we will then take these and make one json.